### Load libraries

In [152]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import tqdm
import sys
import numpy as np
import pandas as pd
import re

### Load helpers

In [6]:
# This may need to change
sys.path.insert(0, "/Users/harrisonchase/workplace/sports/")

from clean_sports_work.sports_reference.api import find_table, extract_table, create_insert_table_sql

### Get data for each year

In [11]:
all_dfs = []
for year in tqdm.tqdm(range(1950, 2020)):
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(year)
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    table_str = find_table(soup, 'advanced_stats')

    yr2018 = extract_table(table_str, header_row=0, get_url=True, start_of_rows=1)
    drop_cols = yr2018.isnull().mean()[lambda x: x == 1].index
    for col in drop_cols:
        del yr2018[col]
    yr2018['year'] = year
    all_dfs.append(yr2018)

100%|██████████| 70/70 [03:47<00:00,  5.48s/it]


In [12]:
all_draft_dfs = []
for year in tqdm.tqdm(range(1950, 2020)):
    url = 'https://www.basketball-reference.com/draft/NBA_{}.html'.format(year)
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    table_str = find_table(soup, 'stats')

    yr2018 = extract_table(table_str, header_row=1, get_url=True, start_of_rows=2)
    drop_cols = yr2018.isnull().mean()[lambda x: x == 1].index
    for col in drop_cols:
        del yr2018[col]
    yr2018['year'] = year
    all_draft_dfs.append(yr2018)

100%|██████████| 70/70 [02:05<00:00,  1.34s/it]


### Rough cleaning of data

In [287]:
all_stats = pd.concat(all_dfs)
all_stats = all_stats.replace('', np.nan)
all_stats = all_stats.dropna(subset=['player_url', 'mp', 'age'])
float_cols = ['bpm', 'ts_pct', 'per', 'usg_pct', 'obpm', 'dbpm', 
              'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct',
             'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'ws', 'ows', 'dws']
for col in float_cols:
    all_stats[col] = all_stats[col].astype(float)
    
int_cols = ['mp', 'age']
for col in int_cols:
    all_stats[col] = all_stats[col].astype(int)

In [288]:
all_stats['pos'] = all_stats['pos'].str.split('-').str[0]

### Draft stats

In [290]:
all_draft = pd.concat(all_draft_dfs).dropna(subset=['player_url'])
all_draft = all_draft.replace('', np.nan)
all_draft = all_draft.drop_duplicates(subset=['player_url'], keep='last')

In [291]:
all_draft['pick_overall'] = all_draft['pick_overall'].astype(float)

In [292]:
all_stats = all_stats.merge(all_draft[['player_url', 'pick_overall', 'college_name']], how='left', on='player_url')

### Create id for each (player, year)

In [293]:
all_stats['id'] = all_stats['player_url'] + '___' + all_stats['year'].astype(str)

### Deduplicate ids

Occurs if player played on multiple teams in one season, take row where team == 'TOT'

In [294]:
multiple_ids = all_stats['id'].value_counts()[lambda x: x> 1].index

In [297]:
changed = all_stats[all_stats['id'].isin(multiple_ids) & (all_stats['team_id'] == 'TOT')]
first_team = all_stats.iloc[changed.index + 1]
if not all(first_team['player_url'].values == changed['player_url'].values):
    raise ValueError
changed['started_team'] = first_team['team_id'].values
no_change = all_stats[~all_stats['id'].isin(multiple_ids)]
no_change['started_team'] = no_change['team_id']
base_all_stats = pd.concat([
    no_change,
    changed,
])

/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### Add in row for next year

Be careful not to use this in your model evaluation!!! Is only present for ease of predicting next year

In [235]:
most_recent_year = base_all_stats[base_all_stats['year'] == base_all_stats['year'].max()]
most_recent_year['year'] += 1
most_recent_year['age'] +=1
most_recent_year['id'] = most_recent_year['player_url'] + '___' + most_recent_year['year'].astype(str)


/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on

In [236]:
def get_team(url):
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    team_p = [x for x in soup.findAll('div', {'id': 'meta'})[0].findAll('p') if x.find('strong') and (x.find('strong').text == 'Team')]

    if len(team_p) == 0:
        return np.nan
    elif len(team_p) == 1:
        return team_p[0].find('a')['href'][7:10]
    else:
        raise ValueError

In [237]:
teams_now = []
for url_suffix in tqdm.tqdm(most_recent_year['player_url']):
    url = 'https://www.basketball-reference.com' + url_suffix
    teams_now.append(get_team(url))


 29%|██▉       | 155/530 [03:26<12:29,  2.00s/it]


 58%|█████▊    | 309/530 [06:36<04:05,  1.11s/it]


 87%|████████▋ | 463/530 [09:50<01:18,  1.17s/it]


100%|██████████| 530/530 [11:24<00:00,  1.24s/it]


In [238]:
most_recent_year['team_id'] = teams_now
most_recent_year['started_team'] = teams_now

/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/harrisonchase/workplace/.venvs/record_linking/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [239]:
most_recent_year = most_recent_year.dropna(subset=['started_team'])

In [240]:
for col in most_recent_year.columns:
    if col not in ['player', 'pos', 'age', 'team_id', 'id', 'player_url', 'pick_overall', 'college_name', 'started_team', 'year']:
        most_recent_year[col] = np.nan

In [355]:
most_recent_year['team_id_url'] = '/teams/' + most_recent_year['team_id'] + '/2020.html'

In [429]:
all_stats = pd.concat([base_all_stats, most_recent_year])

### Rename teams

In [357]:
team_ids = all_stats[all_stats['mp'].notnull()]['team_id'].unique()

In [358]:
team_renamer = dict()
for team_id in set(team_ids).difference({'TOT'}):
    url = 'https://www.basketball-reference.com/teams/{}/'.format(team_id)
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")
    s = soup.find('script').text
    if (s is not None) and ('teams/' in s):
        regexp = re.compile("teams/(.*)/")
        s1 = regexp.search(s).group(1)
        team_renamer[team_id] = s1

In [430]:
all_stats['team_id'] = all_stats['team_id'].apply(team_renamer.get).fillna(all_stats['team_id'])
all_stats['started_team'] = all_stats['started_team'].apply(team_renamer.get).fillna(all_stats['started_team'])

In [433]:
all_stats['started_team_url'] = '/teams/' + all_stats['started_team'] + '/' + all_stats['year'].astype(str) +'.html'

### Add team data

In [411]:
team_ids = all_stats[all_stats['mp'].notnull()]['team_id'].unique()

In [361]:
team_dfs = []
for team_id in set(team_ids).difference({'TOT'}):
    url = 'https://www.basketball-reference.com/teams/{}/'.format(team_id)
    html = urlopen(url)

    # create the BeautifulSoup object
    soup = BeautifulSoup(html, "lxml")

    table_str = find_table(soup, team_id)

    yr2018 = extract_table(table_str, header_row=0, get_url=True, start_of_rows=1)

    drop_cols = set(yr2018.isnull().mean()[lambda x: x == 1].index)
    for col in drop_cols:
        del yr2018[col]

    yr2018['year'] = yr2018['lg_id_url'].str[-9:-5].astype(int)
    team_dfs.append(yr2018)

In [434]:
all_team_df = pd.concat(team_dfs).rename(columns={'team_name_url': 'started_team_url'})
all_team_df['team_id'] = all_team_df['started_team_url'].str[7:10]
all_team_df['team_id'] = all_team_df['team_id'].apply(team_renamer.get).fillna(all_team_df['team_id'])
all_team_df['started_team_url'] = '/teams/' + all_team_df['team_id'] + '/' + all_team_df['year'].astype(str) + '.html'

In [435]:
all_team_df['made_playoffs'] = all_team_df['rank_team_playoffs'].notnull()

In [436]:
all_team_df = all_team_df.replace('', np.nan)

In [437]:
for col in ['win_loss_pct', 'srs', 'pace_rel', 'off_rtg_rel', 'def_rtg_rel', 'made_playoffs']:
    all_team_df[col] = all_team_df[col].astype(float)

In [438]:
filtered_team_df = all_team_df[['started_team_url', 'win_loss_pct', 'srs', 'pace_rel', 'off_rtg_rel', 'def_rtg_rel', 'made_playoffs']]

In [439]:
all_stats = all_stats.merge(filtered_team_df, how='left', on='started_team_url')

### Coach info

In [440]:
coaches_by_year = all_team_df[['started_team_url', 'coaches_url', 'year']].fillna(method='bfill')

In [441]:
coaches_by_year['team_id'] = coaches_by_year['started_team_url'].str[7:10]
coaches_by_year['team_id'] = coaches_by_year['team_id'].apply(team_renamer.get).fillna(coaches_by_year['team_id'])

In [442]:
p_year = coaches_by_year.copy()
p_year['year'] += 1

In [443]:
merged = coaches_by_year.merge(p_year, how='left', on=['team_id', 'year'], suffixes=('', '_p'))
merged = merged.iloc[::-1]

In [444]:
merged['year_coaching'] = 0

In [445]:
prev = np.nan
p_t = np.nan
yr_coaching = 0
for i, row in merged.iterrows():
    if pd.isnull(prev):
        merged.loc[i, 'year_coaching'] = 0
        yr_coaching = 0
    elif (prev == row['coaches_url_p']) and (p_t == row['team_id']):
        yr_coaching +=1
        merged.loc[i, 'year_coaching'] = yr_coaching
    else:
        merged.loc[i, 'year_coaching'] = 0
        yr_coaching = 0
    prev = row['coaches_url_p']
    p_t = row['team_id']

In [446]:
all_stats_team = all_stats.merge(merged[['started_team_url', 'year_coaching']], how='left', on='started_team_url')

### Save data

In [448]:
all_stats_team.to_msgpack('all_stats.mp')

In [449]:
all_stats_team[all_stats_team['year'] == 2017]

,ranker,player,pos,age,team_id,g,mp,per,ts_pct,fg3a_per_fga_pct,...,id,started_team,started_team_url,win_loss_pct,srs,pace_rel,off_rtg_rel,def_rtg_rel,made_playoffs,year_coaching
17484,1,Álex Abrines,SG,23,OKC,68,1055.0,10.1,0.560,0.724,...,/players/a/abrinal01.html___2017,OKC,/teams/OKC/2017.html,0.573,1.14,1.4,-0.5,-1.3,1.0,0.0
17485,3,Steven Adams,C,23,OKC,80,2389.0,16.5,0.589,0.002,...,/players/a/adamsst01.html___2017,OKC,/teams/OKC/2017.html,0.573,1.14,1.4,-0.5,-1.3,1.0,0.0
17486,4,Arron Afflalo,SG,31,SAC,61,1580.0,8.9,0.559,0.360,...,/players/a/afflaar01.html___2017,SAC,/teams/SAC/2017.html,0.390,-3.29,-1.5,-1.5,2.5,1.0,0.0
17487,5,Alexis Ajinça,C,28,NOH,39,584.0,12.9,0.529,0.022,...,/players/a/ajincal01.html___2017,NOH,/teams/NOH/2017.html,0.415,-1.69,1.6,-3.6,-1.4,1.0,0.0
17488,6,Cole Aldrich,C,28,MIN,62,531.0,12.7,0.549,0.000,...,/players/a/aldrico01.html___2017,MIN,/teams/MIN/2017.html,0.378,-0.64,-1.8,2.0,3.2,1.0,0.0
17489,7,LaMarcus Aldridge,PF,31,SAS,72,2335.0,18.6,0.532,0.053,...,/players/a/aldrila01.html___2017,SAS,/teams/SAS/2017.html,0.744,7.13,-2.2,2.3,-5.3,1.0,18.0
17490,8,Lavoy Allen,PF,27,IND,61,871.0,11.6,0.485,0.006,...,/players/a/allenla01.html___2017,IND,/teams/IND/2017.html,0.512,-0.64,-0.5,-0.2,0.0,1.0,4.0
17491,9,Tony Allen,SG,35,MEM,71,1914.0,13.3,0.493,0.091,...,/players/a/allento01.html___2017,MEM,/teams/MEM/2017.html,0.524,0.96,-4.1,-1.1,-1.7,1.0,2.0
17492,10,Al-Farouq Aminu,PF,26,POR,61,1773.0,11.3,0.506,0.455,...,/players/a/aminual01.html___2017,POR,/teams/POR/2017.html,0.500,-0.23,0.3,1.5,2.0,1.0,3.0
17493,11,Chris Andersen,C,38,CLE,12,114.0,11.6,0.497,0.136,...,/players/a/anderch01.html___2017,CLE,/teams/CLE/2017.html,0.622,2.87,-0.2,4.8,1.5,1.0,1.0
